In [2]:
import os
import subprocess
import time
import sys
sys.path.append('/workspace/activation-steering')

In [2]:
# Start Xvfb manually (xvfb-run does this under the hood)
subprocess.Popen([
    'Xvfb', ':99', '-screen', '0', '1280x1024x24', '+extension', 'GLX', '+render', '-noreset'
])

time.sleep(2)  # Give Xvfb time to start

(EE) 
Fatal server error:
(EE) Server is already active for display 99
	If this server is no longer running, remove /tmp/.X99-lock
	and start again.
(EE) 


In [3]:
# Set environment variables to match your working terminal setup
os.environ['DISPLAY'] = ':99'
os.environ['LIBGL_ALWAYS_SOFTWARE'] = '1'
#os.environ['LIBGL_DRIVERS_PATH'] = '/usr/lib/x86_64-linux-gnu/dri' - for google cloud
#os.environ['LD_PRELOAD'] = '/usr/lib/x86_64-linux-gnu/libffi.so.7'

In [4]:
!glxinfo | grep 'OpenGL renderer string'
!glxinfo | grep 'OpenGL version string'
!glxinfo | grep 'GLX version'
!glxinfo | grep 'renderer'

OpenGL renderer string: llvmpipe (LLVM 15.0.7, 256 bits)
OpenGL version string: 4.5 (Compatibility Profile) Mesa 23.2.1-1ubuntu3.1~22.04.3
GLX version: 1.4
    GLX_MESA_copy_sub_buffer, GLX_MESA_query_renderer, GLX_MESA_swap_control, 
    GLX_MESA_query_renderer, GLX_OML_swap_method, GLX_SGIS_multisample, 
Extended renderer info (GLX_MESA_query_renderer):
OpenGL renderer string: llvmpipe (LLVM 15.0.7, 256 bits)


In [5]:
import openai
from arguments import get_config
from interfaces import setup_LMP, LMP_interface
from visualizers import ValueMapVisualizer
from envs.rlbench_env import VoxPoserRLBench
from utils import set_lmp_objects
import numpy as np
from rlbench import tasks
from LMP import LMP

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [6]:
config = get_config('rlbench')
# uncomment this if you'd like to change the language model (e.g., for faster speed or lower cost)
# for lmp_name, cfg in config['lmp_config']['lmps'].items():
#     cfg['model'] = 'gpt-3.5-turbo'

# initialize env and voxposer ui
visualizer = ValueMapVisualizer(config['visualizer'])
env = VoxPoserRLBench(visualizer=visualizer)
print("done visualizer")

QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'


done visualizer


In [6]:
task = env.rlbench_env.get_task(tasks.SetTheTable)
task

## Substituting LLM and testing LMP

In [6]:
from steered_model import SteeredModel
from transformers import AutoModelForCausalLM, AutoTokenizer
from activation_steering import MalleableModel, SteeringVector
from arguments import get_config

In [7]:
steer_cfg = get_config(config_path='./configs/steering.yaml')
steer_cfg.model_name, steer_cfg.cache_dir

('NousResearch/Hermes-2-Pro-Llama-3-8B', '/workspace/.cache/huggingface')

In [7]:
#AutoModelForCausalLM.from_pretrained(steer_cfg.model_name, cache_dir=steer_cfg.cache_dir, device_map="auto")

In [ ]:
controller_config = config['controller']
planner_config = config['planner']
lmp_env_config = config['lmp_config']['env']
lmps_config = config['lmp_config']['lmps']
env_name = config['env_name']

In [8]:
lmp_env = LMP_interface(env, lmp_env_config, controller_config, planner_config, env_name=env_name)

##################################################
## voxel resolution: [0.0105 0.0131 0.01  ]
##################################################




In [9]:
import transforms3d
fixed_vars = {
  'np': np,
  'euler2quat': transforms3d.euler.euler2quat,
  'quat2euler': transforms3d.euler.quat2euler,
  'qinverse': transforms3d.quaternions.qinverse,
  'qmult': transforms3d.quaternions.qmult,
}  # external library APIs
variable_vars = { # maps name of function to the function itself, dir(class) returns all attributes of the class
  k: getattr(lmp_env, k)
  for k in dir(lmp_env) if callable(getattr(lmp_env, k)) and not k.startswith("_")
}  # our custom APIs exposed to LMPs

# allow LMPs to access other LMPs
lmp_names = [name for name in lmps_config.keys() if not name in ['composer', 'planner', 'config']]
lmp_names

['parse_query_obj',
 'get_affordance_map',
 'get_avoidance_map',
 'get_velocity_map',
 'get_rotation_map',
 'get_gripper_map']

In [11]:
k = 'get_affordance_map'

In [12]:
lmps_config[k]

{'config': {'prompt_fname': 'get_affordance_map_prompt',
  'model': 'gpt-4',
  'max_tokens': 512,
  'temperature': 0,
  'query_prefix': '# Query: ',
  'query_suffix': '.',
  'stop': ['# Query: ', 'objects ='],
  'maintain_session': False,
  'include_context': False,
  'has_return': True,
  'return_val_name': 'ret_val',
  'load_cache': True},
 'prompt_fname': 'get_affordance_map_prompt',
 'model': 'gpt-4',
 'max_tokens': 512,
 'temperature': 0,
 'query_prefix': '# Query: ',
 'query_suffix': '.',
 'stop': ['# Query: ', 'objects ='],
 'maintain_session': False,
 'include_context': False,
 'has_return': True,
 'return_val_name': 'ret_val',
 'load_cache': True}

In [14]:

affordance_lmp = LMP(k, lmps_config[k], fixed_vars, variable_vars, True, env_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The X11 connection broke: I/O error (code 1)
XIO:  fatal IO error 2 (No such file or directory) on X server ":99"
      after 3815 requests (3815 known processed) with 0 events remaining.


RuntimeError: unknown parameter type

## Testing New Affordance Maps

In [ ]:
lmps, lmp_env = setup_LMP(env, config, debug=False)

In [ ]:
voxposer_ui = lmps['plan_ui']

## Playground

By default we use one of the instructions that come with each task. However, you may treat each task as simply a scene initialization from RLBench, and feel free to try any task you can come up with for the scene.

Note:
- Whether an instruction can be executed or not depends on 1) whether relevant objects are available, and 2) capabilities of the overall algorithm.
- Each execution may produce one or more visualizations. You may view them in "./visualizations/" folder.
- The prompts are adapted with minimal change from the real-world environment in the VoxPoser paper. If a task failure is due to incorrect code generated by the LLM, feel free to modify the relevant prompt in "./prompts/" folder.
- You may view the reward by printing "env.latest_reward". These are computed by RLBench for each task.
- To inspect in viewer without performing any action, you may call "env.rlbench_env._scene.step()" in a loop.

In [6]:
# # uncomment this to show all available tasks in rlbench
# # NOTE: in order to run a new task, you need to add the list of objects (and their corresponding env names) to the "task_object_names.json" file. See README for more details.
# print([task for task in dir(tasks) if task[0].isupper() and not '_' in task])

In [ ]:
env.load_task(tasks.SetTheTable)

In [ ]:
# below are the tasks that have object names added to the "task_object_names.json" file
# uncomment one to use
#env.load_task(tasks.PutRubbishInBin)
# env.load_task(tasks.LampOff)
# env.load_task(tasks.OpenWineBottle)
# env.load_task(tasks.PushButton)
# env.load_task(tasks.TakeOffWeighingScales)
# env.load_task(tasks.MeatOffGrill)
# env.load_task(tasks.SlideBlockToTarget)
# env.load_task(tasks.TakeLidOffSaucepan)
# env.load_task(tasks.TakeUmbrellaOutOfUmbrellaStand)
env.load_task(tasks.SetTheTable)

descriptions, obs = env.reset()
set_lmp_objects(lmps, env.get_object_names())  # set the object names to be used by voxposer

In [ ]:
descriptions

In [8]:
instruction = np.random.choice(descriptions)
voxposer_ui(instruction)

KeyError: 'use_model'

In [ ]:
env.latest_reward

1.0